In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

In [ ]:
# Read the stroke_diagnosis.csv file
df = pd.read_csv("CSV\imports\o05_30_percent_filled_dataset.csv")

In [ ]:
# Assuming your DataFrame is named df
df = df[df['los'] < 10]

In [ ]:
display (df)

In [ ]:
# Set training percentage. The difference goes to test set
training_percentage = 0.7

# It's already sorted. Just for precaution. Sort by 'subject_id' and 'Time_Zone')
df = df.sort_values(by=['subject_id', 'Time_Zone'])

# Calculate the total number of unique subject IDs
unique_subject_ids = df['subject_id'].nunique()

# Calculate the number of unique subject IDs to include in the training set
train_subject_ids_count = int(training_percentage * unique_subject_ids)

# Initialize variables to track the number of subject IDs included in the training set
subject_ids_in_training = 0

# Initialize empty DataFrames for the training and test sets
train_df = pd.DataFrame(columns=df.columns)
test_df = pd.DataFrame(columns=df.columns)

# Iterate through the sorted DataFrame
for subject_id, subject_data in df.groupby('subject_id'):
    if subject_ids_in_training < train_subject_ids_count:
        # Add this subject's data to the training set
        train_df = pd.concat([train_df, subject_data])
        subject_ids_in_training += 1
    else:
        # Add this subject's data to the test set
        test_df = pd.concat([test_df, subject_data])

# Reset the index of the resulting DataFrames
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# I'm going to use those numbers as the split point in rapidminer filter operator
display("The last row of the training set is -> " + str(train_df.tail(1)["row_count"].values[0]))

In [ ]:
# Concatenate train_df and test_df for consistent encoding of categorical variables
combined_df = pd.concat([train_df, test_df], axis=0)

# Encode categorical variables
categorical_cols = ['gender', 'language', 'marital_status', 'race']
combined_df_encoded = pd.get_dummies(combined_df, columns=categorical_cols)

# Convert 'age' column to numeric type
combined_df_encoded['age'] = pd.to_numeric(combined_df_encoded['age'], errors='coerce')

# Convert 'hospital_expire_flag' column to boolean type
combined_df_encoded['hospital_expire_flag'] = combined_df_encoded['hospital_expire_flag'].astype(bool)


# Split the dataframe at the original row index (before concatenation)
combined_df_encoded_train = combined_df_encoded.iloc[:len(train_df)]
combined_df_encoded_test = combined_df_encoded.iloc[len(train_df):]

# Split data into features and target variable again
X_train = combined_df_encoded_train.drop(['row_count', 'subject_id', 'hadm_id', 'Time_Zone', 'los'], axis=1)
y_train = combined_df_encoded_train['los']
X_test = combined_df_encoded_test.drop(['row_count', 'subject_id', 'hadm_id', 'Time_Zone', 'los'], axis=1)
y_test = combined_df_encoded_test['los']

# Train XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

In [ ]:
# Metrics
print("Mean Square Error (MSE):", mean_squared_error(y_test, y_pred))
print("Mean Absolute Error (MAE):", mean_absolute_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", mean_squared_error(y_test, y_pred, squared=False))

# MSLE calculation must not have negative values in y_test and y_pred
try:
    msle = mean_squared_log_error(y_test, y_pred)
    print("Mean Squared Logarithmic Error (MSLE):", msle)
except ValueError:
    print("Mean Squared Logarithmic Error cannot be calculated because targets contain negative values.")

# Most important features

In [ ]:
# Get feature importances
feature_importance = model.feature_importances_

# Create a DataFrame to store feature importances along with their corresponding names
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importance})

# Sort the DataFrame by feature importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print the top N most important features
top_n = 20  # Change this value to display more or fewer top features
print(f"Top {top_n} most important features:")
print(feature_importance_df.head(top_n))

# Hyperparameter

In [ ]:
"""
Testing field
"""

from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'learning_rate': np.arange(0.01, 0.51, 0.01),  # Learning rate from 0.01 to 0.51 with step 0.01
    'max_depth': np.arange(1, 11, 1),  # Maximum depth from 1 to 10 with step 1
    'lambda': np.arange(0.0, 10.0, 0.1),  # L2 regularization from 0.0 to 10.0 with step 0.1 
    'alpha': np.arange(0.0, 10.0, 0.1),  # L1 regularization from 0.0 to 10.0 with step 0.1 
    #'n_estimators': [100, 200, 300],  # Number of trees in the forest
    #'subsample': [0.7, 0.8, 0.9],  # Subsample ratio of the training instances
    #'colsample_bytree': [0.7, 0.8, 0.9],  # Subsample ratio of columns when constructing each tree
    #'gamma': [0, 0.1, 0.2]  # Minimum loss reduction required to make a further partition on a leaf node
}

# Create a grid search object
grid_search = GridSearchCV(estimator=xgb.XGBRegressor(objective='reg:squarederror'),
                            param_grid=param_grid,
                            cv=5,  # Number of folds in cross-validation
                            scoring='neg_mean_squared_error',  # Scoring metric
                            verbose=1,  # Controls the verbosity
                            n_jobs=-1)  # Number of jobs to run in parallel (-1: all processors)

# Search for best hyperparameters
grid_search.fit(X_train, y_train)

# Best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Best model
best_model = grid_search.best_estimator_

# Predict on test set using the best model
y_pred_best = best_model.predict(X_test)

# Best model evaluation
print("Mean Square Error (MSE):", mean_squared_error(y_test, y_pred_best))
print("Mean Absolute Error (MAE):", mean_absolute_error(y_test, y_pred_best))
print("Root Mean Squared Error (RMSE):", mean_squared_error(y_test, y_pred_best, squared=False))
print("R-squared (R2):", r2_score(y_test, y_pred_best))